# _PublicationName

- Sample Size: 300 - 320 random users

### Twitter Audit Stats _As of June 29, 2019_
- Fake% | Real Followers | Fake Followers
- ##%	| ##,###	 | ##,###

# Import 

In [2]:
import matplotlib
%matplotlib inline

from dotenv import load_dotenv
load_dotenv()

import os
import requests

import json
from pprint import pprint

from bs4 import BeautifulSoup
import pandas as pd

# Opening Botometer JSON File

In [ ]:
#Open JSON File
with open('file.json') as f:
    data = json.load(f)
pprint(data)

In [1]:
#List of all bots usernames
for bot in data:
    if bot['cap']['universal'] > 0.2: #greater than 2
        print(bot['user']['screen_name'])

In [2]:
#Links
# for bot in data:
#     if bot['cap']['universal'] > 0.2: #greater than 2
#         username = bot['user']['screen_name']
#         print(f"https://twitter.com/{username}")

# BeautifulSoup Sraping (Twitter Variables)

In [ ]:
# # TESTING SECTION
# url = "https://twitter.com/...user..."
# print("Now scraping", url)

# #The page
# response = requests.get(url)
# doc = BeautifulSoup(response.text)
# print(doc)

In [ ]:
# Username
print(doc.find(class_="u-linkComplex-target").text.strip())

# Location
print(doc.find(attrs={'data-place-id': True}).text.strip())

# Date Joined
print(doc.find(class_="ProfileHeaderCard-joinDateText").text.strip())

# Biography
print(doc.find('title').findNext('meta').findNext('meta')['content'])

# Following
print(doc.find(class_="ProfileNav-item--following").findNext(class_="ProfileNav-value").text.strip())

# Followers
print(doc.find(class_="ProfileNav-item--followers").findNext(class_="ProfileNav-value").text.strip())

# Building the DataFrame

In [ ]:
rows = []

for bot in data:
    if bot['cap']['universal'] > 0.2:
        username = bot['user']['screen_name']        
        url = f"https://twitter.com/{username}"
        print("Now scraping:", url)

        #Download each page
        response = requests.get(url)
        doc = BeautifulSoup(response.text)

        #Scraping
        row = {}
        try:
            row['Username'] = doc.find(class_="u-linkComplex-target").text.strip()
        except:
            pass
        try:
            row['Location'] = doc.find(attrs={'data-place-id': True}).text.strip()
        except:
            pass
        try:
            row['Join_Date'] = doc.find(class_="ProfileHeaderCard-joinDateText").text.strip()
        except:
            pass
        try:
            row["Bio"] = doc.find('title').findNext('meta').findNext('meta')['content']
        except:
            pass
        try:
            row['Following'] = doc.find(class_="ProfileNav-item--following").findNext(class_="ProfileNav-value").text.strip()
        except:
            pass
        try:
            row['Followers'] = doc.find(class_="ProfileNav-item--followers").findNext(class_="ProfileNav-value").text.strip()
        except:
            pass
        row['Link'] = url
        rows.append(row)

print("Building DataFrame")
df = pd.DataFrame(rows)
df.head()

# Cleaning Data

In [7]:
df['Join_Date'] = df['Join_Date'].str.replace('(Joined )', '', regex = True)
df['Bio'] = df['Bio'].str.replace('(.* from)', '', regex = True)
df['Followers'].fillna(0, inplace=True)
df['Following'].fillna(0, inplace=True)
#set(df['Bio'])
df

In [10]:
df.to_csv("_Publication.csv", index=False)